In [1]:
from tensorflow import keras
import tensorflow as tf
from tensorflow import keras

def conv_block(input_tensor,kernel_size,n_filters,strides,padding='valid'):
    x = keras.layers.Conv2D(
        kernel_size=kernel_size,
        filters=n_filters,
        padding=padding,strides=strides,
        kernel_initializer='he_normal')(input_tensor)
    x = keras.layers.BatchNormalization(axis=1)(x)
    #x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Activation('relu')(x)
    #x = keras.layers.Dropout(0.1)(x)
    return x

In [2]:
def conv_block_di(input_tensor,kernel_size,n_filters,dilation_rate,padding='same'):
    x = keras.layers.Conv2D(
        kernel_size=kernel_size,
        filters=n_filters,dilation_rate=dilation_rate,padding=padding)(input_tensor)
    x = keras.layers.BatchNormalization(axis=1)(x)
    #x = keras.layers.Activation('relu')(x)
    #x = keras.layers.Dropout(0.1)(x)

    #x = keras.layers.Conv2D(
        #kernel_size=(1,1),
        #filters=n_filters,
        #padding='same',
        #kernel_regularizer = keras.regularizers.L2(5e-4),
        #bias_regularizer=keras.regularizers.L2(5e-4))(input_tensor)
    #x = keras.layers.BatchNormalization(axis=1)(x)

    return keras.layers.Activation('relu')(x)

In [4]:
def multiscale_block(input_tensor,n_filters,kernel_size=None):
    tow1 = conv_block(input_tensor=input_tensor,kernel_size=kernel_size,strides=(1,1),n_filters=n_filters)
    print('**********************')
    print(tow1.shape)
    tow2 = conv_block(input_tensor=input_tensor,kernel_size=kernel_size,strides=(1,1),n_filters=n_filters)
    tow2 = conv_block_di(input_tensor=tow2,kernel_size=kernel_size,dilation_rate=(3,3),n_filters=n_filters)
    tow2 = conv_block(input_tensor=tow2,kernel_size=(1,1),strides=(1,1),n_filters=n_filters)
    print(tow2.shape)
    tow3 = conv_block(input_tensor=input_tensor,kernel_size=kernel_size,strides=(1,1),n_filters=n_filters)
    tow3 = conv_block_di(input_tensor=tow3,kernel_size=kernel_size,dilation_rate=(3,3),n_filters=n_filters)
    tow3 = conv_block_di(input_tensor=tow3,kernel_size=kernel_size,dilation_rate=(5,5),n_filters=n_filters)
    tow3 = conv_block(input_tensor=tow3,kernel_size=(1,1),strides=(1,1),n_filters=n_filters)
    print(tow3.shape)
    x = keras.layers.Add()([tow1,tow2,tow3])
    x = keras.layers.BatchNormalization(axis=1)(x)
    x = keras.layers.Activation('relu')(x)
    return x

In [5]:
a = tf.ones((32,8,8,160))

In [11]:
z = multiscale_block(a,160,kernel_size=(5,5))

**********************
(32, 4, 4, 160)
(32, 4, 4, 160)
(32, 4, 4, 160)


In [12]:
x = keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(z)